## Imports
---

In [30]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews as mr
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('sentiwordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is alr

In [31]:
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

import matplotlib.pyplot as plt



import re #Regexs
import os #Acces a fitxers

\

## Exploració de dades
---

In [32]:
# Obtener las categorías disponibles
categories = mr.categories()
print(categories)

# Obtener las IDs de las revisiones de películas
file_ids = mr.fileids()
print(file_ids[8])



['neg', 'pos']
neg/cv008_29326.txt


## Preprocessing
---

In [33]:
def preprocessing(text):
    
    stop_words = set(stopwords.words('english'))
    
    # Tokenización
    text = re.sub(r'[^\w\s]', '', text) # Elimina signos de puntuación
    text = re.sub(r'\d+', '', text) # Elimina números
    text = text.lower() # Convierte a minúsculas
    text = re.sub(r'[\s]+', ' ', text) # Elimina espacios en blanco sobrantes excepto \n
    text = re.sub(r' +', ' ', text) # Elimina espacios en blanco largos
    text = re.sub(r'_', '', text) # Elimina barras bajas

    tokens = nltk.word_tokenize(text)
    
    # Eliminar stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens)

## Train-Test division
---

In [34]:
#Creem la nostra X i la nostra y amb les nostres dades 
X, y = [], []

for elem in file_ids:
    X.append(mr.raw(elem))
    y.append(mr.categories(elem)[0])
    

In [35]:
#dividimos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [36]:
for i, elem in enumerate(X_train):
    X_train[i] = preprocessing(elem)

In [37]:
y_num_train = []

for elem in y_train:
    if elem == 'neg':
        elem = 0
    else:
        elem = 1
    y_num_train.append(elem)

y_train = y_num_train

## Representació
---

- Lematitzar 
- Vectoritzar (fit en el train, transform en el train i en el test) (counts >= 5)
- Modelos


### Lematització i vectorització
---

In [38]:
wnl = nltk.stem.WordNetLemmatizer()

def lemmatize(p):
  d = {'NN': 'n', 'NNS': 'n', 
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}
  if p[1] in d:
    return wnl.lemmatize(p[0], pos=d[p[1]])
  return p[0]

def lem_func(text):
    tokens = nltk.word_tokenize(text)
    tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [39]:
X_train_lemmatized = []

for text in X_train:
    X_train_lemmatized.append(lem_func(text))

In [40]:
cv = CountVectorizer(min_df=5)
cv.fit(X_train_lemmatized)

def vect(text):
    a = cv.transform(text)
    return pd.DataFrame(a.toarray(), columns=cv.get_feature_names_out())


In [41]:
X_train_final = vect(X_train_lemmatized)

In [42]:
X_train_final

,aaron,abandon,abby,abduction,abel,ability,able,ably,aboard,abode,...,zinger,zip,zombie,zombielike,zone,zoo,zoom,zucker,zwick,zwicks
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1596,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1598,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Preprocessem les dades de test


In [43]:
y_num_test = []

for elem in y_test:
    if elem == 'neg':
        elem = 0
    else:
        elem = 1
    y_num_test.append(elem)

y_test = y_num_test

In [44]:
for i, elem in enumerate(X_test):
    X_test[i] = preprocessing(elem)

In [45]:
X_test_lemmatized = []

for text in X_test:
    X_test_lemmatized.append(lem_func(text))

In [46]:
X_test_final = vect(X_test_lemmatized)

### Models
---
Per la part supervisada farem diferents models i els compararem entre ells. Els models seran els següents:
- Regressió logística
- Random Forest
- SVM
- XGBoost
- Red neuronal (MLP)
- EBM

Després evaluarem els models segons els resultats al train i  provarem els millors al test.

In [47]:
#Imports de funcions per fer els models
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression

#### Regressió logística


In [48]:
logistic = LogisticRegression(max_iter=500)
logistic.fit(X_train_final, y_train)

LogisticRegression(max_iter=500)

In [49]:
predictions_logistic = logistic.predict(X_train_final)

In [51]:
train_accuracy_logistic = logistic.score(X_test_final, y_test)
train_accuracy_logistic

0.8525

Intentarem millorar aquests resultats probant diferents hipermarametres i fent cross-validation amb cadascun d'ells utilitzant la funció `GridSearchCV` de `sklearn`.

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter': [100, 200, 300]
}

## PART 2
---

In [52]:
y_num_unsupervised = []

for elem in y:
    if elem == 'neg':
        elem = 0
    else:
        elem = 1
    y_num_unsupervised.append(elem)


In [54]:
def get_synset(word, context):
    # Tokenize the context
    context_tokens = word_tokenize(context)
    
    # Use the Lesk algorithm to get the most likely synset
    synset = lesk(context_tokens, word)
    
    return synset

def get_sentiment_score(synset_tuple):
    total = 0
    
    # Iterate over the synsets in the tuple
    for synset in synset_tuple:
        if synset is None:
            continue
        # Get the SentiSynset for this synset
        senti_synset = swn.senti_synset(synset.name())
        
        # Add the sentiment score for this synset to the total score
        total += senti_synset.pos_score() - senti_synset.neg_score()

    return total

def normalize_sentiment_score(scores):
    
    for score in scores:
        if score <= 0:
            scores[score] = 0
        else:
            scores[score] = 1
        
    


            
    
    
    


In [1]:
text_synsets = []

for text in X:
    # Divide the text into lines
    lines = text.split('\n')
    
    # Iterate over the lines in the text
    for line in lines:
        # Tokenize the line
        tokens = word_tokenize(line)
        
        # Initialize an empty list to hold the synsets for this line
        line_synsets = []
        
        # Iterate over the words in the line
        for word in tokens:
            # Get the synset for the word
            synset = get_synset(word, line)
            
            print(f"word: {word} - {synset}")
            
            # Add the synset to the list of synsets for this line
            line_synsets.append(synset)
        
        # Convert the list of synsets to a tuple and add it to the list of tuples
        text_synsets.append(tuple(line_synsets))


NameError: name 'X' is not defined

In [ ]:

print(text_synsets)

In [ ]:

sentiment_scores = []

# Iterate over the tuples of synsets
for synset_tuple in text_synsets:
    get_sentiment_score(synset_tuple)
    # Add the total score for this tuple to the list of sentiment scores
    sentiment_scores.append(total)
    
    
print(sentiment_scores)


normalize_sentiment_score(sentiment_scores)

print(sentiment_scores)